In [2]:
# 導入所需的模塊
from haystack import Document, Pipeline
from haystack.document_stores.in_memory import InMemoryDocumentStore
from haystack.components.classifiers import DocumentLanguageClassifier
from haystack.components.routers import MetadataRouter
from haystack.components.writers import DocumentWriter

/Users/samhsiao/Documents/PythonVenv/envHaystack/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# 準備各種語言的酒店評論樣本
documents = [
    Document(content="Super appartement. Juste au dessus de plusieurs bars qui ferment très tard. A savoir à l'avance. (Bouchons d'oreilles fournis !)"),
    Document(content="El apartamento estaba genial y muy céntrico, todo a mano. Al lado de la librería Lello y De la Torre de los clérigos. Está situado en una zona de marcha, así que si vais en fin de semana , habrá ruido, aunque a nosotros no nos molestaba para dormir"),
    Document(content="The keypad with a code is convenient and the location is convenient. Basically everything else, very noisy, wi-fi didn't work, check-in person didn't explain anything about facilities, shower head was broken, there's no cleaning and everything else one may need is charged."),
    Document(content="It is very central and appartement has a nice appearance (even though a lot IKEA stuff), *W A R N I N G the appartement presents itself as a elegant and as a place to relax, very wrong place to relax - you cannot sleep in this appartement, even the beds are vibrating from the bass of the clubs in the same building - you get ear plugs from the hotel -> now I understand why -> I missed a trip as it was so loud and I could not hear the alarm next day due to the ear plugs.- there is a green light indicating 'emergency exit' just above the bed, which shines very bright at night - during the arrival process, you felt the urge of the agent to leave as soon as possible. - try to go to 'RVA clerigos appartements' -> same price, super quiet, beautiful, city center and very nice staff (not an agency)- you are basically sleeping next to the fridge, which makes a lot of noise, when the compressor is running -> had to switch it off - but then had no cool food and drinks. - the bed was somehow broken down - the wooden part behind the bed was almost falling appart and some hooks were broken before- when the neighbour room is cooking you hear the fan very loud. I initially thought that I somehow activated the kitchen fan"),
    Document(content="Un peu salé surtout le sol. Manque de service et de souplesse"),
    Document(content="Nous avons passé un séjour formidable. Merci aux personnes , le bonjours à Ricardo notre taxi man, très sympathique. Je pense refaire un séjour parmi vous, après le confinement, tout était parfait, surtout leur gentillesse, aucune chaude négative. Je n'ai rien à redire de négative, Ils étaient a notre écoute, un gentil message tout les matins, pour nous demander si nous avions besoins de renseignement et savoir si tout allait bien pendant notre séjour."),
    Document(content="Céntrico. Muy cómodo para moverse y ver Oporto. Edificio con terraza propia en la última planta. Todo reformado y nuevo. Te traen un estupendo desayuno todas las mañanas al apartamento. Solo que se puede escuchar algo de ruido de la calle a primeras horas de la noche. Es un zona de ocio nocturno. Pero respetan los horarios.")
]

In [4]:
# 創建各語言的內存文件存儲：英語、法語、西班牙語
en_document_store = InMemoryDocumentStore()
fr_document_store = InMemoryDocumentStore()
es_document_store = InMemoryDocumentStore()

# 創建語言分類器
language_classifier = DocumentLanguageClassifier(
    languages=["en", "fr", "es"]
)

# 創建元數據路由器，根據語言將文件路由到對應的寫入器
router_rules = {
    "en": {"language": {"$eq": "en"}},
    "fr": {"language": {"$eq": "fr"}},
    "es": {"language": {"$eq": "es"}}
}
router = MetadataRouter(rules=router_rules)
# 輸出看一下
router

Inputs:
  - documents: List[Document]
Outputs:
  - unmatched: List[Document]
  - en: List[Document]
  - fr: List[Document]
  - es: List[Document]

In [5]:
# 創建語言專用的寫入器
en_writer = DocumentWriter(document_store=en_document_store)
fr_writer = DocumentWriter(document_store=fr_document_store)
es_writer = DocumentWriter(document_store=es_document_store)

In [6]:
# 創建管道
indexing_pipeline = Pipeline()
# 添加組件
indexing_pipeline.add_component(
    instance=language_classifier, name="language_classifier"
)
indexing_pipeline.add_component(
    instance=router, name="router"
)
indexing_pipeline.add_component(
    instance=en_writer, name="en_writer"
)
indexing_pipeline.add_component(
    instance=fr_writer, name="fr_writer"
)
indexing_pipeline.add_component(
    instance=es_writer, name="es_writer"
)

# 連接組件
indexing_pipeline.connect("language_classifier", "router")
indexing_pipeline.connect("router.en", "en_writer")
indexing_pipeline.connect("router.fr", "fr_writer")
indexing_pipeline.connect("router.es", "es_writer")

🚅 Components
  - language_classifier: DocumentLanguageClassifier
  - router: MetadataRouter
  - en_writer: DocumentWriter
  - fr_writer: DocumentWriter
  - es_writer: DocumentWriter
🛤️ Connections
  - language_classifier.documents -> router.documents (List[Document])
  - router.en -> en_writer.documents (List[Document])
  - router.fr -> fr_writer.documents (List[Document])
  - router.es -> es_writer.documents (List[Document])

In [7]:
# 繪製管道圖
indexing_pipeline.draw("indexing_pipeline.png")

In [8]:
# 運行管道，顯示寫入每個語言的文件數
indexing_pipeline.run(
    data={"language_classifier": {"documents": documents}}
)

{'router': {'unmatched': []},
 'en_writer': {'documents_written': 2},
 'fr_writer': {'documents_written': 3},
 'es_writer': {'documents_written': 2}}

In [9]:
print(
    "English documents: ",
    en_document_store.filter_documents()
)
print(
    "French documents: ",
    fr_document_store.filter_documents()
)
print(
    "Spanish documents: ",
    es_document_store.filter_documents()
)

English documents:  [Document(id=8f64ab234c6a5d5652d02bed144d069ec6e988903b071d16fffbf400abfc1047, content: 'The keypad with a code is convenient and the location is convenient. Basically everything else, very...', meta: {'language': 'en'}), Document(id=1e65b29e0cdbf3263ccb2fa6807288c40eb48b37f04fd37e131e2759c9a017cd, content: 'It is very central and appartement has a nice appearance (even though a lot IKEA stuff), *W A R N I ...', meta: {'language': 'en'})]
French documents:  [Document(id=ea7ea338874232de2d8105a258813f50345db82772e21ad2c4549dbb7adce8a3, content: 'Super appartement. Juste au dessus de plusieurs bars qui ferment très tard. A savoir à l'avance. (Bo...', meta: {'language': 'fr'}), Document(id=6b64c8a60543ee32b81cd39bc8d6e09fae4bff1b22c6ccdcf414db26fa354e7a, content: 'Un peu salé surtout le sol. Manque de service et de souplesse', meta: {'language': 'fr'}), Document(id=b1be23526f19a8af80a190e775bfd05e65878e585529037cb45b47267a4eaa98, content: 'Nous avons passé un séjour fo

In [12]:
from getpass import getpass
import os
from dotenv import load_dotenv

# 導入環境變數
load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
# 判斷是否寫入，若無則手動提供
if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass("Enter OpenAI API key:")

In [13]:
# 導入 RAG 管道所需的組件
from haystack.components.retrievers.in_memory import InMemoryBM25Retriever
from haystack.components.joiners import DocumentJoiner
from haystack.components.builders import PromptBuilder
from haystack.components.generators import OpenAIGenerator
from haystack.components.routers import TextLanguageRouter

In [16]:
# 定義提示模板
prompt_template = """
您將收到有關住宿的評論。
僅根據給定的評論簡潔地回答問題。
評論：
{% for doc in documents %}
    {{ doc.content }}
{% endfor %}
問題：{{ query}}
答案：
"""

In [17]:
# 創建 RAG 管道
rag_pipeline = Pipeline()
rag_pipeline.add_component(
    instance=TextLanguageRouter(["en", "fr", "es"]),
    name="router"
)
rag_pipeline.add_component(
    instance=InMemoryBM25Retriever(document_store=en_document_store),
    name="en_retriever"
)
rag_pipeline.add_component(
    instance=InMemoryBM25Retriever(document_store=fr_document_store),
    name="fr_retriever"
)
rag_pipeline.add_component(
    instance=InMemoryBM25Retriever(document_store=es_document_store),
    name="es_retriever"
)
rag_pipeline.add_component(
    instance=DocumentJoiner(),
    name="joiner"
)
rag_pipeline.add_component(
    instance=PromptBuilder(template=prompt_template),
    name="prompt_builder"
)
rag_pipeline.add_component(
    instance=OpenAIGenerator(),
    name="llm"
)

In [18]:
# 繪製管道圖
rag_pipeline.draw("rag_pipeline.png")